# 支持向量机
支持向量机(Support Vector Machines, SVM): 是一种监督学习算法。

-   支持向量(Support Vector)就是离分隔超平面最近的那些点。
-   机(Machine)就是表示一种算法，而不是表示机器。

## 工作原理
1.  寻找最大分类间距
2.  转而通过拉格朗日函数求优化的问题
    -   数据可以通过画一条直线就可以将它们完全分开，这组数据叫线性可分(linearly separable)数据，而这条分隔直线称为分隔超平面(separating hyperplane)。
    -   如果数据集上升到1024维呢？那么需要1023维来分隔数据集，也就说需要N-1维的对象来分隔，这个对象叫做超平面(hyperlane)，也就是分类的决策边界。

## 寻找最大间隔
### 点到超平面的距离

- 分隔超平面 函数问题: $y(x) = w^T x + b$
- 分类的结果: $f(x) = sign(w^T x + b)$ (sign表示 $>0$ 为1，$<0$ 为-1，$=0$ 为0)
- 点到超平面的几何问题: $d(x) = \frac{(w^T x + b)}{||w||}$  ($||w||$ 表示 $w$ 矩阵的二范数 $=> \sqrt{w^T \cdot w}$, 点到超平面的距离也是类似的)

### 点到直线的距离公式
$$
d = \frac{|Ax_0 + By_0 + C|}{\sqrt{A^2 + B^2}}
$$

### 拉格朗日乘子法
- 类别标签用 $-1, 1$，是为了后期方便 $label \cdot (w^T x + b)$ 的标识和距离计算；如果 $label \cdot (w^T x + b) > 0$ 表示预测正确，否则预测错误。

- 现在目标明确，就是要找到 $w$ 和 $b$，因此我们必须找到最小间隔的数据点，也就是前面所说的支持向量。
    - 也就是，最小的距离取最大值（最小的距离：就是最小间隔的数据点；最大：就是最大间隔；为了找出最优超平面-最终要的就是支持向量）
    - 目标函数：$\arg\max_{w, b} \left(\min [label \cdot (w^T x + b)] * \frac{1}{||w||}\right)$
    - 如果 $label \cdot (w^T x + b) > 0$ 表示预测正确，也称为函数间隔。$||w||$ 可以理解为归一化，也称几何间隔。
    - 比如，令 $label \cdot (w^T x + b) >= 1$，因为 $0-1$之间，得到的点是存在误判的可能性，所以要求 $\min [label \cdot (w^T x + b) = 1]$，才能找到距离更接近的支持向量。

- 结论：找到最优间隔（线性可分）超平面$\arg\max_{w, b} \frac{1}{||w||}$，约束条件就是 $label \cdot (w^T x + b) = 1$

- 新的目标函数求解：$\arg\max_{w, b} \frac{1}{||w||}$
    - => 就是求：$\arg\min_{w, b} ||w||$ （求矩阵会比较麻烦，如果只是 $\frac{1}{2} x^2$ 的偏导数，那么…同样是求最小值）
    - 也就是求：$\arg\min_{w, b} \frac{1}{2} ||w||^2$（二次函数求极值，平方方便计算）
    - 本质上就是线性不等式的二次优化问题（求分隔超平面，等价于求解相应的凸二次规划问题）

- 通过拉格朗日乘子法，求二次优化问题
    - 假设需要求极值的目标函数 (objective function) 为 $f(x,y)$，限制条件为 $\varphi(x,y)=M$
    - 设 $g(x,y)=M-\varphi(x,y)$ 表示下文中 $label \cdot (w^T x + b)$
    - 定义一个新函数: $F(x,y,\lambda)=f(x,y)+\lambda g(x,y)$
    - $\alpha$ 为 $\lambda$ ($\alpha>=0$)，代表要引入的拉格朗日乘子 (Lagrange multiplier)
    - 那么: $\mathcal{L}(w,b,\alpha) = \frac{1}{2} ||w||^2 + \sum_{i=1}^n \alpha_i [1 - label \cdot (w^T x + b)]$
    - 因为 $label \cdot (w^T x + b) \geq 1$，$\alpha \geq 0$，所以 $\alpha [1 - label \cdot (w^T x + b)] \leq 0$，$\sum_{i=1}^n \alpha_i [1 - label \cdot (w^T x + b)] \leq 0$
    - 当 $label \cdot (w^T x + b) > 1$ 则 $\alpha = 0$，表示该点为非支持向量
    - 相当于求解: $\max_{w,b} \mathcal{L}(w,b,\alpha) = \frac{1}{2} ||w||^2$
    - 如果要求: $\min_{w,b} \frac{1}{2} ||w||^2$，也就是要求: $\min_{w,b} (\max_{\alpha} L(w,b,\alpha))$

- 现在转化到对偶问题的求解
    - $\min_{w,b} (\max_{\alpha} L(w,b,\alpha)) \geq \max_{\alpha} (\min_{w,b} L(w,b,\alpha))$
    - 现在分两步:
        - 先求: $\min_{w,b} L(w,b,\alpha) = \frac{1}{2} ||w||^2 + \sum_{i=1}^n \alpha_i [1 - label \cdot (w^T x + b)]$
        - 就是求 $L(w,b,\alpha)$ 关于 $w, b$ 的偏导数，得到 $w$ 和 $b$ 的值，并化简为: 拉格朗日乘子的方程。

- 终于得到了课本上的公式: 
$$
\max_{\alpha} \left( \sum_{i=1}^m \alpha_i - \frac{1}{2} \sum_{i,j=1}^m label_i \cdot label_j \cdot \alpha_i \cdot \alpha_j \cdot \langle x_i, x_j \rangle \right)
$$
- 约束条件: $\alpha \geq 0$ 并且 $\sum_{i=1}^m \alpha_i \cdot label_i = 0$

在支持向量机（SVM）中，松弛变量（Slack Variable）是为了处理**线性不可分**情况下的一种扩展，常用于软间隔（Soft Margin）SVM。在线性不可分的情况下，简单的线性超平面无法将所有数据正确分类。引入松弛变量能够允许一些数据点在一定范围内违背分类规则，同时保证模型的可解性和对异常点的鲁棒性。

### 松弛变量的引入

SVM的目标是找到一个最大化分类间隔的超平面。对于线性可分的情况，我们的目标是通过最小化 $\frac{1}{2}||w||^2$ 来找到最优的分隔超平面，同时满足约束条件：

$$
label_i \cdot (w^T x_i + b) \geq 1
$$

但是当数据线性不可分时，这个约束无法完全满足。为了允许某些数据点出现分类错误或者不完全满足上述约束条件，我们引入松弛变量 $\xi_i$ 来表示每个数据点的“松弛程度”，即：

$$
label_i \cdot (w^T x_i + b) \geq 1 - \xi_i
$$

其中 $\xi_i \geq 0$，表示每个点的偏差。当 $\xi_i = 0$ 时，数据点正确分类且在间隔边界之外；当 $\xi_i > 0$ 时，数据点允许“侵入”间隔甚至错误分类。

### 软间隔SVM的优化目标

引入松弛变量后，优化问题的目标函数不仅要最小化 $||w||^2$，同时还要控制松弛变量的总和。为此，目标函数变为：

$$
\min \frac{1}{2} ||w||^2 + C \sum_{i=1}^m \xi_i
$$

这里，$C$ 是一个正的常数，它控制模型在最大化间隔与最小化分类错误之间的权衡。如果 $C$ 较大，模型会对分类错误更加敏感，倾向于减少松弛变量的值，试图让更多的数据点分类正确；如果 $C$ 较小，模型会允许更多的分类错误，间隔则会更大。

### 带松弛变量的对偶问题

通过引入松弛变量，SVM的对偶问题中的目标函数变为：

$$
\max_{\alpha} \left( \sum_{i=1}^m \alpha_i - \frac{1}{2} \sum_{i,j=1}^m label_i \cdot label_j \cdot \alpha_i \cdot \alpha_j \cdot \langle x_i, x_j \rangle \right)
$$

同时，约束条件也发生了变化：
- $\alpha_i \geq 0$
- $\sum_{i=1}^m \alpha_i \cdot label_i = 0$
- $0 \leq \alpha_i \leq C$，其中 $C$ 是控制松弛变量的上限。

新增的约束 $0 \leq \alpha_i \leq C$ 反映了松弛变量的影响。如果 $C$ 越大，意味着松弛变量可以增大，允许更多的分类错误；如果 $C$ 越小，表示模型要求严格分类，更少容忍错误。


## SMO算法
**序列最小优化（Sequential Minimal Optimization, SMO）** 是一种用于训练支持向量机（SVM）的算法。SMO 是解决 SVM 中二次规划问题的一种高效方法，通过分解问题为两个变量的子问题来简化计算，避免了传统 SVM 中大规模矩阵求解的复杂度。

SMO 的主要贡献是显著减少了每一步的计算开销，因此适合处理大规模数据集。下面我将详细讲解 SMO 算法的核心思想和步骤。

### 背景

在训练 SVM 时，我们需要求解如下的优化问题：

#### 优化目标：
$$
\min_{w,b,\xi} \frac{1}{2} ||w||^2 + C \sum_{i=1}^m \xi_i
$$

#### 约束条件：
$$
label_i (w^T x_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0
$$

这个优化问题在其对偶形式下表现为：

#### 对偶问题的目标函数：
$$
\max_{\alpha} \left( \sum_{i=1}^m \alpha_i - \frac{1}{2} \sum_{i,j=1}^m \alpha_i \alpha_j label_i label_j \langle x_i, x_j \rangle \right)
$$

#### 约束条件：
$$
0 \leq \alpha_i \leq C, \quad \sum_{i=1}^m \alpha_i label_i = 0
$$

其中，$\alpha_i$ 是拉格朗日乘子，$C$ 是惩罚参数。传统求解 SVM 二次规划问题的方法通常需要大规模矩阵运算，计算复杂度较高。而 SMO 算法通过简化计算过程，使其可以快速求解。

### SMO 的核心思想

SMO 算法的核心思想是**分解**：在每一步中，选择两个拉格朗日乘子 $\alpha_i$ 和 $\alpha_j$，并保持其他拉格朗日乘子固定不变。这样一来，问题就被简化为一个关于两个变量的二次规划问题，容易求解。为什么只选择两个变量？因为对于两个变量的二次规划问题，我们可以直接找到解析解，而不需要使用复杂的数值优化方法。

### SMO 算法的步骤

1. **选择变量 $\alpha_i$ 和 $\alpha_j$：**

   - 每次迭代，SMO 需要选择两个拉格朗日乘子 $\alpha_i$ 和 $\alpha_j$ 进行更新。
   - 通常，第一个变量 $\alpha_i$ 是违反 KKT 条件（Karush-Kuhn-Tucker）的样本点（这些点即是需要优化的点）。
   - 第二个变量 $\alpha_j$ 是根据 $\alpha_i$ 的选择来确定的，通常选择使得更新的效果最大化。

2. **计算更新的上下界：**

   由于 $\alpha_i$ 和 $\alpha_j$ 受到约束条件 $\sum_{i=1}^m \alpha_i label_i = 0$ 的限制，因此这两个变量之间存在某种依赖关系。通过这个约束条件，可以计算出这两个变量的更新范围（即上下界）。

   上下界为：

   $$
   L = \max(0, \alpha_j - \alpha_i), \quad H = \min(C, C + \alpha_j - \alpha_i)
   $$

3. **更新 $\alpha_i$ 和 $\alpha_j$：**

   利用二次规划中的闭式解更新 $\alpha_i$ 和 $\alpha_j$：

   $$
   \alpha_j^{new} = \alpha_j + \frac{label_j \cdot (E_i - E_j)}{\eta}
   $$

   其中，$E_i$ 和 $E_j$ 分别是第 $i$ 和第 $j$ 个数据点的预测误差，$\eta$ 是一个计算的常量。$\alpha_i$ 的更新则根据 $\alpha_j$ 的更新直接得到。

4. **更新偏置项 $b$：**

   由于每次迭代 $\alpha_i$ 和 $\alpha_j$ 都发生变化，偏置项 $b$ 也需要更新。通常的更新方式如下：

   $$
   b_1 = b - E_i - label_i (\alpha_i^{new} - \alpha_i) \langle x_i, x_i \rangle - label_j (\alpha_j^{new} - \alpha_j) \langle x_i, x_j \rangle
   $$

   $$
   b_2 = b - E_j - label_i (\alpha_i^{new} - \alpha_i) \langle x_i, x_j \rangle - label_j (\alpha_j^{new} - \alpha_j) \langle x_j, x_j \rangle
   $$

   然后 $b$ 的更新取决于新的 $\alpha_i$ 和 $\alpha_j$ 是否满足 $0 < \alpha_i^{new} < C$ 和 $0 < \alpha_j^{new} < C$ 的条件。如果都满足，则更新为 $b_1$ 或 $b_2$，否则通过加权方式计算。

5. **重复迭代直到收敛：**

   SMO 算法会不断重复上述步骤，直到所有的 $\alpha_i$ 均满足 KKT 条件，算法收敛为止。

### SMO 的优点

1. **快速处理大规模数据**：SMO 避免了对整个拉格朗日乘子集合的全局优化，而是通过分解为两个变量的子问题大大加快了计算速度。其复杂度远低于传统的二次规划求解方法。
  
2. **简化内存使用**：由于每次只优化两个拉格朗日乘子，SMO 不需要像其他算法那样构建和存储大的二次规划矩阵，因此对内存的要求较低。

3. **可扩展性**：SMO 可以很好地扩展到非线性核函数，处理线性不可分的数据问题，兼顾了线性和非线性 SVM 的求解。

### 总结

SMO 算法通过每次仅更新两个拉格朗日乘子，将大规模 SVM 的优化问题分解为小规模的二次规划问题，从而加速了训练过程。它能够处理大规模数据集，特别是在内存有限的情况下具有明显优势。



In [7]:
import numpy as np


def loadDataSet(fileName):
    dataMat = []
    labelMat = []

    with open(fileName, 'r') as f:
        for line in f.readlines():
            lineArr = line.strip().split('\t')
            dataMat.append([float(lineArr[0]), float(lineArr[1])])
            labelMat.append(float(lineArr[2]))
    
    return dataMat, labelMat

# 简易SMO的数学原理
简化版的序列最小优化算法（Simplified Sequential Minimal Optimization，简化SMO）是用于训练支持向量机（SVM）的一种高效算法。它通过在每次迭代中仅优化两个拉格朗日乘子（即两个 $\alpha$ 值），将复杂的二次规划问题分解为一系列简单的二次优化子问题。下面，我将详细讲解简化SMO算法的数学原理和过程。

## 一、支持向量机的优化问题

首先，我们从SVM的基本优化问题开始。

对于给定的训练数据集 $\{(x_i, y_i)\}_{i=1}^m$，其中 $x_i \in \mathbb{R}^n$，$y_i \in \{-1, +1\}$，我们希望找到一个决策函数：

$$
f(x) = \text{sgn}(w^Tx + b)
$$

其中，$w$ 是权重向量，$b$ 是偏置项。

### 1. 原始优化问题

SVM的原始优化问题是：

$$
\begin{align*}
\min_{w, b, \xi} \quad & \frac{1}{2} \|w\|^2 + C \sum_{i=1}^m \xi_i \\
\text{s.t.} \quad & y_i(w^Tx_i + b) \geq 1 - \xi_i, \quad \xi_i \geq 0, \quad i = 1, \dots, m
\end{align*}
$$

其中，$\xi_i$ 是松弛变量，$C$ 是惩罚参数。

### 2. 对偶问题

通过引入拉格朗日乘子 $\alpha_i$ 和 $\mu_i$，可以将原始问题转换为对偶问题：

$$
\begin{align*}
\max_{\alpha} \quad & W(\alpha) = \sum_{i=1}^m \alpha_i - \frac{1}{2} \sum_{i=1}^m \sum_{j=1}^m \alpha_i \alpha_j y_i y_j x_i^T x_j \\
\text{s.t.} \quad & 0 \leq \alpha_i \leq C, \quad \sum_{i=1}^m \alpha_i y_i = 0
\end{align*}
$$

此时，权重向量 $w$ 和偏置 $b$ 可以表示为：

$$
w = \sum_{i=1}^m \alpha_i y_i x_i
$$

## 二、简化SMO算法的核心思想

简化SMO算法的核心在于每次只选择两个拉格朗日乘子 $\alpha_i$ 和 $\alpha_j$ 进行优化，而固定其他 $\alpha$ 值。这使得高维的优化问题简化为在二维空间中的优化问题，极大地降低了计算复杂度。

### 1. 选择优化变量

在每次迭代中，算法需要选择违反KKT条件的 $\alpha_i$ 和另一个 $\alpha_j$ 进行优化。选择 $\alpha_i$ 的原则是其对应的样本未满足KKT条件，即：

- 如果 $\alpha_i = 0$，则要求 $y_i f(x_i) \geq 1$
- 如果 $0 < \alpha_i < C$，则要求 $y_i f(x_i) = 1$
- 如果 $\alpha_i = C$，则要求 $y_i f(x_i) \leq 1$

其中，$f(x_i) = w^T x_i + b$。

### 2. 优化两个 $\alpha$ 值

由于对偶问题的约束条件 $\sum_{i=1}^m \alpha_i y_i = 0$，当 $\alpha_i$ 和 $\alpha_j$ 变化时，需要满足：

$$
\alpha_i y_i + \alpha_j y_j = \text{常数}
$$

因此，在调整 $\alpha_i$ 和 $\alpha_j$ 时，可以将其中一个表示为另一个的函数。

#### (1) 计算误差

首先，计算当前样本的预测值和误差：

$$
E_i = f(x_i) - y_i = \left( \sum_{k=1}^m \alpha_k y_k x_k^T x_i + b \right) - y_i
$$

#### (2) 计算 $\eta$

$\eta$ 是用于更新 $\alpha_j$ 的关键变量，定义为：

$$
\eta = 2 x_i^T x_j - x_i^T x_i - x_j^T x_j
$$

需要注意，如果 $\eta \geq 0$，则停止本次迭代，因为此时无法保证目标函数的下降。

#### (3) 更新 $\alpha_j$

根据优化目标和约束条件，新的 $\alpha_j$ 可以表示为：

$$
\alpha_j^{\text{new}} = \alpha_j^{\text{old}} + \frac{y_j (E_i - E_j)}{\eta}
$$

然后，需要将 $\alpha_j^{\text{new}}$ 裁剪到合法范围 $[L, H]$ 内。

##### 计算上下界 $L$ 和 $H$

根据约束条件 $\alpha_i y_i + \alpha_j y_j = \text{常数}$，可以推导出 $\alpha_j$ 的取值范围：

- 如果 $y_i \neq y_j$：

  $$
  L = \max(0, \alpha_j^{\text{old}} - \alpha_i^{\text{old}})
  $$
  
  $$
  H = \min(C, C + \alpha_j^{\text{old}} - \alpha_i^{\text{old}})
  $$

- 如果 $y_i = y_j$：

  $$
  L = \max(0, \alpha_j^{\text{old}} + \alpha_i^{\text{old}} - C)
  $$
  
  $$
  H = \min(C, \alpha_j^{\text{old}} + \alpha_i^{\text{old}})
  $$

将计算得到的 $\alpha_j^{\text{new}}$ 裁剪到 $[L, H]$ 范围内：

$$
\alpha_j^{\text{new}} = \begin{cases}
H, & \text{if } \alpha_j^{\text{new}} > H \\
\alpha_j^{\text{new}}, & \text{if } L \leq \alpha_j^{\text{new}} \leq H \\
L, & \text{if } \alpha_j^{\text{new}} < L
\end{cases}
$$

#### (4) 更新 $\alpha_i$

由于 $\alpha_i y_i + \alpha_j y_j = \text{常数}$，因此：

$$
\alpha_i^{\text{new}} = \alpha_i^{\text{old}} + y_i y_j (\alpha_j^{\text{old}} - \alpha_j^{\text{new}})
$$

#### (5) 计算偏置项 $b$

更新 $\alpha_i$ 和 $\alpha_j$ 后，需要更新偏置 $b$。新的偏置可以通过以下公式计算：

$$
b_1 = b - E_i- y_i (\alpha_i^{\text{new}} - \alpha_i^{\text{old}}) x_i^T x_i - y_j (\alpha_j^{\text{new}} - \alpha_j^{\text{old}}) x_i^T x_j
$$

$$
b_2 = b - E_j - y_i (\alpha_i^{\text{new}} - \alpha_i^{\text{old}}) x_i^T x_j - y_j (\alpha_j^{\text{new}} - \alpha_j^{\text{old}}) x_j^T x_j
$$

根据 $\alpha_i^{\text{new}}$ 和 $\alpha_j^{\text{new}}$ 是否在 0 和 $C$ 之间，选择 $b$ 的更新值：

- 如果 $0 < \alpha_i^{\text{new}} < C$，则 $b = b_1$
- 如果 $0 < \alpha_j^{\text{new}} < C$，则 $b = b_2$
- 否则，$b = \frac{b_1 + b_2}{2}$

### 3. 重复迭代

以上步骤需要在整个数据集上反复迭代，直到满足停止条件：

- 达到最大迭代次数
- 所有 $\alpha$ 值都不再发生更新（即所有样本都满足KKT条件）

## 三、算法流程总结

1. **初始化**：将所有 $\alpha_i = 0$，$b = 0$。

2. **外循环**：遍历数据集中的所有样本点，寻找违反KKT条件的样本。

3. **内循环**：对于每个违反KKT条件的 $\alpha_i$，随机选择另一个 $\alpha_j$，并共同进行优化。

4. **计算误差**：计算 $E_i$ 和 $E_j$。

5. **计算上下界**：根据 $\alpha_i$ 和 $\alpha_j$ 的值，计算 $L$ 和 $H$。

6. **计算 $\eta$**：计算 $\eta = 2 x_i^T x_j - x_i^T x_i - x_j^T x_j$。

7. **更新 $\alpha_j$**：根据公式更新 $\alpha_j$，并裁剪到 $[L, H]$ 范围内。

8. **更新 $\alpha_i$**：根据约束条件更新 $\alpha_i$。

9. **更新偏置 $b$**：计算 $b_1$ 和 $b_2$，并更新 $b$。

10. **检查收敛**：如果在一轮循环中没有 $\alpha$ 值发生更新，则增加迭代计数器；否则，重置计数器。

11. **结束条件**：当迭代计数器超过最大迭代次数时，算法结束。

## 四、算法的数学原理

### 1. 二次规划问题的简化

SMO算法的关键在于将高维的二次规划问题简化为两个变量的二次规划问题。对于两个变量的二次规划问题，可以通过解析方法直接求解，无需使用复杂的优化算法。

### 2. 保证优化过程的收敛

通过选择违反KKT条件的样本进行优化，算法保证了每次迭代都会使目标函数值增加或保持不变。此外，由于SVM的对偶问题是凸优化问题，算法最终一定会收敛到全局最优解。

### 3. 使用KKT条件作为优化准则

KKT条件在凸优化中起着重要的作用。通过检查每个样本是否满足KKT条件，可以确定哪些 $\alpha$ 需要被优化。

### 4. 维护 $\alpha$ 的约束条件

在优化过程中，始终保证 $\alpha_i$ 和 $\alpha_j$ 满足约束条件：

- $0 \leq \alpha_i, \alpha_j \leq C$
- $\alpha_i y_i + \alpha_j y_j = \text{常数}$

通过裁剪 $\alpha_j$ 和更新 $\alpha_i$，确保了所有 $\alpha$ 始终在合法范围内。

## 五、结论

简化SMO算法通过每次只优化两个拉格朗日乘子，将复杂的二次规划问题简化为一系列简单的子问题，极大地提高了SVM训练的效率。算法利用了KKT条件、对偶问题的凸性和拉格朗日乘子的约束条件，保证了优化过程的正确性和收敛性。


In [ ]:
# import random


# def selectJrand(i, m):
#     """
#     随机选择一个整数
#     Args:
#         i  第一个alpha的下标
#         m  所有alpha的数目
#     Returns:
#         j  返回一个不为i的随机数，在0~m之间的整数值
#     """
#     j = i
#     while j == i:
#         j = int(random.uniform(0, m))
#     return j

# def clipAlpha(aj, H, L):
#     """clipAlpha(调整aj的值，使aj处于 L<=aj<=H)
#     Args:
#         aj  目标值
#         H   最大值
#         L   最小值
#     Returns:
#         aj  目标值
#     """
#     if aj > H:
#         aj = H
#     if L > aj:
#         aj = L
#     return aj

# # 简化版SMO算法
# def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
#     """smoSimple
#     Args:
#         dataMatIn    数据集
#         classLabels  类别标签
#         C   松弛变量(常量值)，允许有些数据点可以处于分隔面的错误一侧。
#             控制最大化间隔和保证大部分的函数间隔小于1.0这两个目标的权重。
#             可以通过调节该参数达到不同的结果。
#         toler   容错率（是指在某个体系中能减小一些因素或选择对某个系统产生不稳定的概率。）
#         maxIter 退出前最大的循环次数
#     Returns:
#         b       模型的常量值
#         alphas  拉格朗日乘子
#     """
#     dataMatrix = np.mat(dataMatIn)
#     # np.mat() 默认生成的是 row-matrix，所以需要转置一下
#     labelMat = np.mat(classLabels).T
#     m, n = np.shape(dataMatrix)
#     # 初始化b 和 alphas
#     b = 0
#     alphas = np.mat(np.zeros((m, 1)))

#     # 没有任何alpha改变的情况下遍历数据的次数
#     iter = 0
#     while (iter < maxIter):
#         # 记录alpha是否已经进行优化，每次循环时设为0，然后再对整个集合顺序遍历
#         alphaPairsChanged = 0
#         for i in range(m):
#             # 我们预测的类别 y = w^Tx[i]+b; 其中因为 w = Σ(1~n) a[n]*lable[n]*x[n]
#             # alphas: (m, 1)
#             # labelMat: (m, 1)
#             # multiply(alphas, labelMat): (m, 1)
#             # multiply(alphas, labelMat).T: (1, m)
#             # dataMatrix: (m, n)
#             # dataMatrix[i, :]: (1, n)
#             # dataMatrix[i, :].T: (n, 1)
#             # dataMatrix * dataMatrix[i, :].T: (m, 1)
#             # multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T): (1, 1)
#             # fXi: scalar (float)
#             fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b
#             Ei = fXi - float(labelMat[i])

#             # 约束条件 (KKT条件是解决最优化问题的时用到的一种方法。我们这里提到的最优化问题通常是指对于给定的某一函数，求其在指定作用域上的全局最小值)
#             # 0<=alphas[i]<=C，但由于0和C是边界值，我们无法进行优化，因为需要增加一个alphas和降低一个alphas。
#             # 表示发生错误的概率: labelMat[i]*Ei 如果超出了 toler， 才需要优化。至于正负号，我们考虑绝对值就对了。
#             '''
#             检验训练样本(xi, yi)是否满足KKT条件
#             yi*f(i) >= 1 and alpha = 0 (outside the boundary)
#             yi*f(i) == 1 and 0<alpha< C (on the boundary)
#             yi*f(i) <= 1 and alpha = C (between the boundary)
#             '''

#             # 如果 Ei 满足优化的条件，我们就通过随机选择另外一个拉格朗日乘子alphaj，然后同时优化这两个乘子
#             if ((labelMat[i] * Ei < -toler) and alphas[i] < C) or ((labelMat[i] * Ei > toler) and alphas[i] > 0):
#                 j = selectJrand(i, m)
#                 fXj = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j, :].T)) + b
#                 Ej = fXj - float(labelMat[j])
#                 alphaIold = alphas[i].copy()
#                 alphaJold = alphas[j].copy()

#                 # L和H用于将alphas[j]调整到0-C之间。如果L==H，就不做任何改变，直接执行continue语句
#                 # labelMat[i] != labelMat[j] 表示异侧，就相减，否则是同侧，就相加。
#                 if (labelMat[i] != labelMat[j]):
#                     L = max(0, alphas[j] - alphas[i])
#                     H = min(C, C + alphas[j] - alphas[i])
#                 else:
#                     L = max(0, alphas[j] + alphas[i] - C)
#                     H = min(C, alphas[j] + alphas[i])
#                 # 如果相同，就没法优化了
#                 if L == H:
#                     print("L==H")
#                     continue

#                  # eta是alphas[j]的最优修改量，如果eta==0，需要退出for循环的当前迭代过程
#                 # 参考《统计学习方法》李航-P125~P128<序列最小最优化算法>
#                 eta = 2.0 * dataMatrix[i, :]*dataMatrix[j, :].T - dataMatrix[i, :]*dataMatrix[i, :].T - dataMatrix[j, :]*dataMatrix[j, :].T
#                 if eta >= 0:
#                     print("eta>=0")
#                     continue

#                 # 计算出一个新的alphas[j]值
#                 alphas[j] -= labelMat[j]*(Ei - Ej)/eta
#                 # 并使用辅助函数，以及L和H对其进行调整
#                 alphas[j] = clipAlpha(alphas[j], H, L)
#                 # 检查alpha[j]是否只是轻微的改变，如果是的话，就退出for循环。
#                 if (abs(alphas[j] - alphaJold) < 0.00001):
#                     print("j not moving enough")
#                     continue
#                 # 然后alphas[i]和alphas[j]同样进行改变，虽然改变的大小一样，但是改变的方向正好相反
#                 alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
#                 # 在对alpha[i], alpha[j] 进行优化之后，给这两个alpha值设置一个常数b。
#                 # w= Σ[1~n] ai*yi*xi => b = yj- Σ[1~n] ai*yi(xi*xj)
#                 # 所以:   b1 - b = (y1-y) - Σ[1~n] yi*(a1-a)*(xi*x1)
#                 # 为什么减2遍？ 因为是 减去Σ[1~n]，正好2个变量i和j，所以减2遍
#                 b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[i, :].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i, :]*dataMatrix[j, :].T
#                 b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[j, :].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j, :]*dataMatrix[j, :].T
#                 if (0 < alphas[i]) and (C > alphas[i]):
#                     b = b1
#                 elif (0 < alphas[j]) and (C > alphas[j]):
#                     b = b2
#                 else:
#                     b = (b1 + b2)/2.0
#                 alphaPairsChanged += 1
#                 print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
#         # 在for循环外，检查alpha值是否做了更新，如果在更新则将iter设为0后继续运行程序
#         # 知道更新完毕后，iter次循环无变化，才推出循环。
#         if (alphaPairsChanged == 0):
#             iter += 1
#         else:
#             iter = 0
#         print("iteration number: %d" % iter)
#     return b, alphas

# # def calcWs(alphas, dataArr, classLabels):
# #     """calcWs 基于alpha计算w值
# #     Args:
# #         alphas        拉格朗日乘子
# #         dataArr       feature数据集
# #         classLabels   目标变量数据集
# #     Returns:
# #         wc  回归系数
# #     """
# #     X = np.mat(dataArr)
# #     labelMat = np.mat(classLabels).T
# #     m, n = np.shape(X)
# #     w = np.zeros((n, 1))
# #     for i in range(m):
# #         w += np.multiply(alphas[i]*labelMat[i], X[i, :].T)
# #     return w

# if __name__ == "__main__":
#     # 获取特征和目标变量
#     dataArr, labelArr = loadDataSet('testSet.txt')
#     # print labelArr

#     # b是常量值， alphas是拉格朗日乘子
#     b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)
#     print('/n/n/n')
#     print('b=', b)
#     print('alphas[alphas>0]=', alphas[alphas > 0])
#     print('shape(alphas[alphas > 0])=', np.shape(alphas[alphas > 0]))
#     for i in range(100):
#         if alphas[i] > 0:
#             print(dataArr[i], labelArr[i])


C:\Users\26271\AppData\Local\Temp\ipykernel_7068\3161992511.py:73: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b
C:\Users\26271\AppData\Local\Temp\ipykernel_7068\3161992511.py:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Ei = fXi - float(labelMat[i])
C:\Users\26271\AppData\Local\Temp\ipykernel_7068\3161992511.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fXj = float(np.

L==H
iter: 0 i:1, pairs changed 1
j not moving enough
iter: 0 i:3, pairs changed 2
iter: 0 i:4, pairs changed 3
L==H
L==H
L==H
L==H
L==H
j not moving enough
iter: 0 i:25, pairs changed 4
L==H
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
iter: 0 i:55, pairs changed 5
j not moving enough
iteration number: 0
j not moving enough
iter: 0 i:3, pairs changed 1
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
iter: 0 i:36, pairs changed 2
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
L==H
L==H
j not moving enough
iteration number: 0
j not moving enough
iter: 0 i:1, pairs changed 1
j not moving enough
L==H
j not moving enough
L==H
L==H
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
iter: 0 i:39, pairs changed 2
j not moving enough
L==H
j not moving en

## 核函数


在支持向量机（Support Vector Machine，SVM）中，核函数（Kernel Function）是一种用于处理非线性问题的重要工具。SVM的基本思想是找到一个线性分类器，但现实中很多数据并不能线性分离，因此引入了核函数来实现非线性映射。核函数的主要作用是通过将输入数据映射到高维空间，使得在高维空间中找到一个超平面来进行线性分离，进而解决非线性问题。

### 核函数的定义与作用
核函数 $K(x_i, x_j)$ 实际上是定义在输入数据对 $(x_i, x_j)$ 上的一个函数，它表示输入数据在高维空间中映射后的内积。核函数的主要作用是避免显式地计算映射到高维空间后的特征值，通过核函数直接在低维空间进行运算来获得相同的效果，这称为“核技巧”（Kernel Trick）。SVM模型中只需用核函数替代点积即可实现这种非线性映射。

### 常用的核函数类型
1. **线性核（Linear Kernel）**  
   $$
   K(x_i, x_j) = x_i^T x_j
   $$
   线性核通常用于数据线性可分的情况，适合线性分类问题。

2. **多项式核（Polynomial Kernel）**  
   $$
   K(x_i, x_j) = (x_i^T x_j + c)^d
   $$
   多项式核函数通过参数 $d$ 控制映射维度的多项式次数，使SVM能够处理具有非线性关系的数据。

3. **高斯核 / 径向基核（RBF Kernel）**  
   $$
   K(x_i, x_j) = \exp\left(-\frac{\|x_i - x_j\|^2}{2\sigma^2}\right)
   $$
   高斯核是一种常用的核函数，适合处理复杂的非线性数据。它通过参数 $\sigma$ 控制样本之间的相似度范围。

4. **Sigmoid核（Sigmoid Kernel）**  
   $$
   K(x_i, x_j) = \tanh(\alpha x_i^T x_j + c)
   $$
   Sigmoid核与神经网络的激活函数相似，可以看作是将SVM转化为一种类似神经网络的模型。

### 核函数的选择
核函数的选择直接影响模型的分类能力。一般而言：
- **线性核** 适用于数据呈线性关系的情况。
- **多项式核** 在数据分布较复杂但仍有规律时使用。
- **高斯核** 在数据没有明显的线性或多项式特征时，较为普遍。
- **Sigmoid核** 在小数据集上可能有用，但在大数据集上常不如其他核函数表现稳定。

### 核函数的优缺点
- **优点**：核函数使SVM能够高效处理非线性问题，并且避免了显式计算高维映射。
- **缺点**：核函数的选择和参数调整较为复杂，通常需要通过交叉验证进行调参。此外，复杂核函数计算量大，可能增加模型的计算开销。

### 核技巧的数学解释
对于一个输入样本 $x$ 通过某种非线性映射 $\phi(x)$ 到高维空间，直接在该空间计算较为复杂。通过核函数的定义，可以利用核函数表示：
$$
K(x_i, x_j) = \phi(x_i)^T \phi(x_j)
$$
这样，SVM只需计算核函数 $K(x_i, x_j)$ 即可，而无需显式求出 $\phi(x)$ 的表达，从而大大简化了运算。


In [ ]:
import numpy as np
import os
from os import listdir
import random

def img2vector(filename):
    # 读取文件并将数据转为二维数组
    data = np.genfromtxt(filename, dtype=int, delimiter=1)
    # 返回一维数组
    return np.reshape(data, (1, -1))

def loadImages(dirName):
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = np.zeros((m, 1024))

    for i, fileNameStr in enumerate(trainingFileList):
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])

        hwLabels.append(-1 if classNumStr == 9 else 1)
        trainingMat[i, :] = img2vector(os.path.join(dirName, fileNameStr))
    
    return trainingMat, hwLabels

def loadDataSet(fileName):
    dataMat = []
    labelMat = []

    with open(fileName) as f:
        for line in f:
            lineArr = line.strip().split('\t')
            dataMat.append([float(lineArr[0]), float(lineArr[1])])
            labelMat.append(float(lineArr[2]))
    
    return np.array(dataMat), np.array(labelMat)

In [ ]:
class optStruct:
    """
    建立数据结构来保存所有的重要值
    """
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        """
        Args:
            dataMatIn    数据集
            classLabels  类别标签
            C   松弛变量(常量值)，允许有些数据点可以处于分隔面的错误一侧。
                控制最大化间隔和保证大部分的函数间隔小于1.0这两个目标的权重。
                可以通过调节该参数达到不同的结果。
            toler   容错率
            kTup    包含核函数信息的元组
        """

        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler

        self.m = dataMatIn.shape[0]
        self.alphas = np.zeros((self.m, 1))
        self.b = 0

        # 误差缓存，第一列给出的是eCache是否有效的标志位，第二列给出的是实际的E值。
        self.eCache = np.zeros((self.m, 2))

        # 创建一个m*m的核矩阵，每一列包含了给定数据集的一个核函数计算结果
        self.K = np.zeros((self.m, self.m))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)

def kernelTrans(X, A, KTup):
    """
    核转换函数
    Args:
        X     数据集矩阵
        A     数据集中的单个数据点
        kTup  核函数的信息 (类型和参数)

    Returns:
        K     核转换后的结果矩阵
    """

    m, n = X.shape
    K = np.zeros((m, 1))

    if KTup[0] == 'lin':
        # 线性核
        # linear kernel:   m*n * n*1 = m*1
        K = X @ A.T

    elif KTup[0] == 'rbf':
        deltaRow = X - A
        squaredDistance = np.sum(deltaRow ** 2, axis=1).reshape(-1, 1)
        K = np.exp(squaredDistance / (-1 * KTup[1] ** 2))
    
    else:
        raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    
    return K

In [ ]:
def calcEk(oS, k):
    """calcEk（求 Ek误差: 预测值-真实值的差）

    该过程在完整版的SMO算法中出现次数较多，因此将其单独作为一个方法
    Args:
        oS  optStruct对象
        k   具体的某一行

    Returns:
        Ek  预测结果与真实结果比对，计算误差Ek
    """

    # 在SVM中，每个样本x_i对最终的决策结果的贡献是 alpha_i * y_i * kernel(x_i, x)，alpha_i是拉格朗日乘子, y_i是类别标签
    fXk = np.dot(oS.alphas * oS.labelMat, oS.K[:, k]) + oS.b
    Ek = fXk - oS.labelMat[k]
    return Ek

In [ ]:
def selectJrand(i, m):
    """
    随机选择一个整数
    Args:
        i  第一个alpha的下标
        m  所有alpha的数目
    Returns:
        j  返回一个不为i的随机数，在0~m之间的整数值
    """
    j = random.choice([x for x in range(m) if x != i])
    return j

In [ ]:
def smoP(dataMatIn, classLabels, C, toler, matIter, KTup=('lin', 0)):
    """
    完整的 SMO 算法外循环，用于支持向量机优化。
    
    Args:
        dataMatIn: 数据集矩阵
        classLabels: 类别标签
        C: 松弛变量，控制最大化间隔与容忍误差的权重
        toler: 容错率
        maxIter: 最大迭代次数
        kTup: 包含核函数信息的元组，默认为线性核

    Returns:
        b: 模型的常量值
        alphas: 拉格朗日乘子
    """

    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).T, C, toler, KTup)

    iter = 0